# EIMS Data
This script will read in the individual EIMS data files from the given directory and compile/prase the data for the model.

In [ ]:
source('Scripts - EIMS.r')

# File Input

First we need to set the directory and file name criteria of the data to load. We also set the date-time format that the parser will be expecting.

In [ ]:
input.dir = './Raw Data/EIMS/'
dt.format = "%m/%d/%y %I:%M %p"

## Generate list of files in the directory
file.list = list.files(input.dir)

## split file.list into usable lists
file.list.nu = file.list[grepl('not usable', file.list)] # not usable
file.list.2 = file.list[grepl('corr.txt', file.list)] # usable

## Print total number of files in each category
print(paste('Number of files in file.list:', length(file.list)))
print(paste('Number of files in file.list.2:', length(file.list.2)))
print(paste('Number of files in file.list.nu:', length(file.list.nu)))


In [ ]:
## Read in first data file and configure formatting around it
data = read.csv(paste0(input.dir, file.list.2[1]), sep='\t')
data$time = strptime(data$time, format = dt.format, tz='GMT')

## Run the parser
This will initialize the parser for valve = 1 (sampling) and then valve = 2 (calibration). Included in this is the ___filter.eims()___ and ___take.avg()___ functions.

__NB This will take a while...__

In [ ]:
data.v1 = read.eims(1, input.dir, file.list.2, N = 2, verbose = FALSE, template = data, make.img = TRUE)

In [ ]:
data.v2 = read.eims(2, input.dir, file.list.2, N = 2, verbose = FALSE, template = data, make.img = FALSE)

In [ ]:
## Also, correct the time since the year is not actually 2021...
data.v1$time = data.v1$time - 4 * 365.25 * 86400 - 86400
data.v2$time = data.v2$time - 4 * 365.25 * 86400 - 86400

## Remove NA O2:Ar values
data.v1 = data.v1[which(!is.na(data.v1$O2.Ar)),]
data.v2 = data.v2[which(!is.na(data.v2$O2.Ar)),]

## Number of entries, do they seem right?
nrow(data.v1)
length(which(data.v1$Flag == 3))
nrow(data.v2)
length(which(data.v2$Flag == 3))

## Review results

In [ ]:
#pdf('./Output/Generate-EIMS-review1.pdf')

par(mfrow=c(2,1))
cols = c('black', 'green', 'red')

plot(data.v1$time, data.v1$O2.Ar, pch=16, cex=0.3, ylab='O2/Ar', xlab='', col=cols[data.v1$Flag])
points(data.v2$time, data.v2$O2.Ar, pch=16, cex=0.2, col=cols[data.v2$Flag])

legend(data.v1$time[1], 19, c('Measurements', 'Calibration'), col=c('black', 'green'), pch=16, cex=0.6)

plot(data.v1$time, data.v1$N2.Ar, pch=16, cex=0.3, ylab='N2/Ar', xlab='', col=cols[data.v1$Flag])
points(data.v2$time, data.v2$N2.Ar, pch=16, cex=0.2, col=cols[data.v2$Flag])

plot(data.v1$time, data.v1$time, pch=16, cex=0.3, ylab='Time', xlab='', col=cols[data.v1$Flag])
points(data.v2$time, data.v2$time+86400, cex=0.2, col=cols[data.v2$Flag])

plot(data.v1$time, data.v1$O2..uM., pch=16, cex=0.3, ylab='O2', xlab='', col=cols[data.v1$Flag])
points(data.v2$time, data.v2$O2..uM., pch=16, cex=0.2, col=cols[data.v2$Flag])

legend(data.v2$time[1], 470, c('Measurements', 'Calibration'), col=c('black', 'green'), pch=16, cex=0.6)

#dev.off()

In [ ]:
#pdf('./Output/Generate-EIMS-review2.pdf')

plot(data.v1$time, data.v1$time, pch=16, cex=0.3, ylab='Time', xlab='', col = cols[data.v1$Flag])
points(data.v2$time, data.v2$time+86400/3, pch=16, cex=0.3, col = cols[data.v2$Flag])
legend(data.v1$time[1], data.v1$time[nrow(data.v1)], c('Measurements', 'Calibration', 'Reject'),
       col=cols, pch=16)

#dev.off()

In [ ]:
plot(data.v1$time, data.v1$TP, col='white', ylim=c(3.2e-6, 5.5e-6), ylab='Total Pressure', xlab='')

col = c('Black', 'blue', 'grey', 'dark green')

for (i in 1:length(unique(data.v1[,1]))) {
    l = which(data.v1[,1] == unique(data.v1[,1])[i])
    points(data.v1$time[l], data.v1$TP[l], col=col[i %% 2 + 1], pch=20, cex=0.6)
}

for (i in 1:length(unique(data.v2[,1]))) {
    l = which(data.v2[,1] == unique(data.v2[,1])[i])
    points(data.v2$time[l], data.v2$TP[l], col=col[i %% 2 + 3], pch=20, cex=0.6)
}


# Save

In [ ]:
## Save the data
eims = list(sample = data.v1, cal = data.v2)
save(eims, file='./RStates/EIMS.2min.2min.rdata')

## Optional
#write.csv(data.v1, './Input Data/EIMS-Valve1.2min.csv')
#write.csv(data.v2, './Input Data/EIMS-Valve2.60min.csv')

In [ ]:
load('./RStates/EIMS.2min.2min.rdata')

In [ ]:
par(mfrow=c(3,1))
#pdf('Output/EIMS-24H Plots.pdf')
eims.v1 = eims.v1[!is.na(eims.v1$time),]
dt = floor(as.numeric(difftime(eims.v1$time, min(eims.v1$time), units='days')))
dt2 = floor(as.numeric(difftime(eims.v2$time, min(eims.v2$time, na.rm = TRUE), units='days')))

for (i in 1:floor(as.numeric(difftime(max(eims.v1$time), min(eims.v1$time), units='days')))) {
    k = which(dt < i & dt >= i-1)
    if(length(k) < 1) {
        k = 1
    }
    k2 = which(dt2 < i & dt2 >= i-1)
    plot(eims.v1$time[k], eims.v1$O2.Ar[k], pch=20, main=paste('Day -', i), xlab='', ylab='O2/Ar', ylim=c(20,30))
    points(eims.v2$time[k2], eims.v2$O2.Ar[k2], pch=20, col='red')
}
#dev.off()